## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.8.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [37]:
messages.reset_index(inplace=True)

In [38]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [39]:
import nltk
import re
from nltk.corpus import stopwords

In [40]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()


In [42]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [43]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [44]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4830, 2325, 2496, 2700, 298, 2809, 117, 3840, 1089, 889],
 [4741, 3660, 2993, 1244, 3870, 3168, 3176],
 [4923, 3741, 1587, 4349],
 [646, 1436, 718, 3981, 4875, 1695],
 [2095, 3870, 1474, 1384, 1602, 2656, 3870, 1080, 2992, 301],
 [1671,
  3939,
  3314,
  1403,
  2335,
  3792,
  2299,
  1182,
  1942,
  3174,
  2913,
  4984,
  1003,
  192,
  3176],
 [3938, 491, 375, 929, 919, 3828, 782, 1409, 4011, 845, 1025],
 [2618, 2821, 3572, 102, 213, 1925, 3792, 736, 4011, 845, 1025],
 [3473, 3584, 50, 4123, 3231, 4812, 1624, 1500, 3792, 738],
 [261, 1063, 1877, 3103, 1603, 3730, 1030, 1388],
 [1198, 1214, 3161, 1506, 1860, 3918, 1863, 1772, 1193, 4686, 614],
 [3981, 4150, 298, 4812, 3792, 213],
 [3404, 889, 3865, 4014, 2588, 2341, 890, 998, 4396],
 [1469, 4874, 4434, 1650, 3756, 723, 3152, 4011, 845, 1025],
 [879, 627, 818, 4761, 185, 4011, 845, 1025],
 [3826, 122, 2076, 449, 3696, 947, 3781, 2800, 1403, 742],
 [2989, 3687, 3660],
 [4392, 2969, 3475, 2226, 3792, 645, 747, 3176],
 [4817, 1589, 29

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr[1]

[4741, 3660, 2993, 1244, 3870, 3168, 3176]

### Embedding Representation

In [48]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4830 2325 2496 ...    0    0    0]
 [4741 3660 2993 ...    0    0    0]
 [4923 3741 1587 ...    0    0    0]
 ...
 [4785  717 3726 ...    0    0    0]
 [2021  213 3223 ...    0    0    0]
 [1211 3304 3362 ...    0    0    0]]


In [49]:
embedded_docs[1]

array([4741, 3660, 2993, 1244, 3870, 3168, 3176,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [50]:
embedded_docs[0]

array([4830, 2325, 2496, 2700,  298, 2809,  117, 3840, 1089,  889,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [51]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
len(embedded_docs),y.shape

(18285, (18285,))

In [53]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [54]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [56]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 19ms/step - loss: 0.3026 - accuracy: 0.8630 - val_loss: 0.1955 - val_accuracy: 0.9186
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1335 - accuracy: 0.9478 - val_loss: 0.2104 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0977 - accuracy: 0.9629 - val_loss: 0.2235 - val_accuracy: 0.9122
Epoch 4/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0716 - accuracy: 0.9743 - val_loss: 0.2582 - val_accuracy: 0.9095
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0538 - accuracy: 0.9830 - val_loss: 0.3204 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0431 - accuracy: 0.9852 - val_loss: 0.3794 - val_accuracy: 0.9070
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0326 - accuracy: 0.9900 - val_loss: 0.4524 - val_accuracy: 0.9082

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [57]:
y_pred=model.predict(X_test)

In [58]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [59]:
from sklearn.metrics import confusion_matrix

In [60]:
confusion_matrix(y_test,y_pred)

array([[3159,  260],
       [ 323, 2293]], dtype=int64)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9033968516984259

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

